# Automated Identification of Vertebral Fractures Using Artificial Intelligence Convolutional Neural Networks Predicts Incident Non-vertebral and Hip Fractures: A Registry-Based Cohort Study

Source code for the paper "Automated Identification of Vertebral Fractures Using Artificial Intelligence Convolutional Neural Networks Predicts Incident Non-vertebral and Hip Fractures: A Registry-Based Cohort Study".

To run the code in your own environment you will require a python 3.6 environment with the following packages installed (more recent versions may work but you'll be on your own):

  * h5py 2.7.1
  * numpy 1.15.4
  * matplotlib 3.0.2
  * opencv-python 4.1.0.25
  * pandas 0.23.4
  * Keras 2.1.5
  * scikit-learn 0.19.1
  * tensorflow-gpu 1.6.0


## Package imports and notebook-wide constants

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

from keras import backend as K
from keras import callbacks, optimizers
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.layers import GlobalAveragePooling2D, Dense, InputLayer, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.models import Model, Sequential, load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import auc, confusion_matrix, roc_curve

%matplotlib inline

In [ ]:
batch_size = 12
height = 600
width = int(height * 0.6)

base = '/data/VFA'
train_dir = os.path.join(base, 'phase_1_all_balanced_up')
valid_dir = os.path.join(base, 'phase_2_all')

## Load previous model

Use these cells to load and verify a previously trained model.

In [ ]:
model = load_model('incresnetv2_600_sgdr_1_10_epochs_all_final.h5')

In [ ]:
model.summary()

## Create new model

Create new network model using a base model of Inception-ResNet-v2 with a 'top' of global average pooling and two dense layers for classification. Random initialization is used throughout.

In [ ]:
model = Sequential()
base_model = InceptionResNetV2(weights=None, include_top=False)
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
base_model.summary()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=optimizers.Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

## Image augmentation and data generators

Keras includes a nice set of functionality for sampling and augmenting model training data. In this instance we'll use Keras' ImageDataGenerator to randomly apply selected transformations to the base data when training. The transformations used in model training are:

  * Rotation by up to 30 degrees
  * Shifting the image horizontally by up to 20% of the image width
  * Shifting the image vertically by up to 20% of the image height
  * Shearing the image by up to 0.2 radians
  * Increasing/reducing color channel intensity by up to 10%

Points outside the original image boundaries (e.g. from shearing) are filled with a constant value of 0, i.e. black. 
Images are multipled (rescaled in Keras parlance) by 1 / 255 to bring pixel values into the range \[0, 1\] before any other transformations are applied. 

Test and validation data will not be transformed beyond rescaling of pixel values into the range \[0, 1\].

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30, 
    zoom_range=0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    channel_shift_range=0.1,
    fill_mode='constant',
    cval=0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width), 
    shuffle=True, 
    batch_size=batch_size,
    class_mode='binary',
    classes=['norm', 'frac'])

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
    valid_dir,
    target_size=(height, width),
    shuffle=False, 
    batch_size=batch_size,
    class_mode='binary',
    classes=['norm', 'frac'])

## Visually check image augmentation

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = [16, 16]

train_frac = os.path.join(train_dir, 'frac')
fnames = [os.path.join(train_frac, fname) for fname in os.listdir(train_frac)]
img_path = fnames[4]
img = image.load_img(img_path, target_size=(height, width))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 1

for batch in train_datagen.flow(x, batch_size=1):
    plt.subplot(1, 4, i)
    plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

## Class weights

Here we set training weights for the classes to account for the imbalance in the number of examples of each class (no fracture, fracture) in the training data. In our case, the number of examples of the negative class (0, normal) is approximately 4x that of the positive class (1, fracture). Weighting the positive cases by their frequency of occurence relative to the negative case guides the training algorithm to converge to a point that minimizes the network loss (error) for postive cases at potentially the expense of accuracy for negative cases.

In [ ]:
train_labels, train_counts = np.unique(train_generator.classes, return_counts=True)
class_weight = {0: 1., 1: train_counts[0] / train_counts[1]}

## Medical metrics callback


A Keras callback to calculate and report medically relevant model metrics at the completion of each training epoch (complete run through the training data set). Metrics include:

  * the false postive rate, true positive rate, and thresholds for the receiver operating characteristic;
  * the area under the curve;
  * the confusion matrix (true/false positives, true/false negatives);
  * the sensitive and specificity of the model;
  * the positive and negative predictive values;
  * postive and negative likelihood ratios; and
  * overall accuracy.


In [ ]:
def med_metrics(labels, predictions, thresh = 0.5):
  
    md = {}

    md['fpr'], md['tpr'], md['th'] = roc_curve(labels, predictions)
    md['auc'] = auc(md['fpr'], md['tpr'])

    thresh_pred = np.copy(predictions)
    thresh_pred[predictions >= thresh] = 1
    thresh_pred[predictions < thresh] = 0

    md['cm'] = confusion_matrix(labels, thresh_pred)
    md['tn'], md['fp'], md['fn'], md['tp'] = md['cm'].ravel()
    md['sn'] = md['tp'] / (md['tp'] + md['fn'])
    md['sp'] = md['tn'] / (md['tn'] + md['fp'])
    md['snspavg'] = (md['sn'] + md['sp']) / 2
    md['ppv'] = md['tp'] / (md['tp'] + md['fp'])
    md['npv'] = md['tn'] / (md['tn'] + md['fn'])
    md['plr'] = md['sn'] / (1 - md['sp'])
    md['nlr'] = md['sp'] / (1 - md['sn'])
    md['acc'] = (md['tp'] + md['tn']) / (md['tp'] + md['tn'] + md['fp'] + md['fn'])

    return md

In [ ]:
class MedMetricsCallback(callbacks.Callback):
    def on_train_begin(self, logs = {}):
        self.hist = {'fpr': [],
                     'tpr': [],
                     'th': [],
                     'auc': [],
                     'cm': [],
                     'tn': [],
                     'fp': [],
                     'fn': [],
                     'tp': [],
                     'sn': [],
                     'sp': [],
                     'snspavg': [],
                     'ppv': [],
                     'npv': [],
                     'plr': [],
                     'nlr': [],
                     'acc': []
                    }
        
    def on_epoch_end(self, epoch, logs = {}):
        self.val_pred = model.predict_generator(validation_generator)
        self.md = med_metrics(validation_generator.classes, self.val_pred, thresh=0.5)
        
        for key, value in self.md.items():
            self.hist[key].append(value)
        
        print('\nVal Sn:', round(self.md['sn'], 3),
              'Sp:', round(self.md['sp'], 3),
              'Avg:', round(self.md['snspavg'], 3),
              'Acc:', round(self.md['acc'], 3),
              'AUC:', round(self.md['auc'], 3), 
              '\n')

In [ ]:
mmetrics = MedMetricsCallback()

## SGDR callback by epoch


Here we define a set of functions and Keras callback to implement a stochastic gradient descent with restarts (SGDR) using a simple cosine-based learning rate decay with a reset schedule. 

In [ ]:
# Returns a simple, single cosine-based learning rate annealing schedule falling to nearly zero over the 
# specified number of steps
def cos_sgdr(steps_per_cycle):
  sched = np.arange(0, 1, 1 / steps_per_cycle)
  sched = np.pi * sched
  sched = np.cos(sched)
  sched = (sched + 1) / 2
  return list(sched)

# Returns a list of learning rates for multiple cycles; exp=1 makes all cycles same length, exp>1 causes 
# them to lengthen
def cos_sgdr_sched(steps_per_cycle, cycles, exp):
  sched = []

for cycle in range(cycles):
    subsched = []
    subsched = cos_sgdr(steps_per_cycle * (exp**cycle))
    for value in subsched: sched.append(value)
        
  return sched

In [ ]:
# Schedule for use with learning rate adjustment after each epoch, rather than by batch
sgdr_sched_epoch = cos_sgdr_sched(10, 1, 2) 
plt.plot(range(len(sgdr_sched_epoch)), sgdr_sched_epoch)
print('Schedule length =', len(sgdr_sched_epoch))

In [ ]:
max_lr = 10**-4 # rescale learning rate schedule to max learning rate from finder (below)
sgdr_sched_epoch = np.asarray(sgdr_sched_epoch)
sgdr_sched_epoch = sgdr_sched_epoch * max_lr
print('Starting learning rate =', sgdr_sched_epoch[0])

In [ ]:
def sgdr_sched_cb_fxn(epoch):
  return sgdr_sched_epoch[epoch]

sgdr_sched_cb = callbacks.LearningRateScheduler(sgdr_sched_cb_fxn, verbose=1)

## Training models

Here we train the network using combinations of class weighting / no class weighting, and SGDR / no SGDR. When working with the notebook you should choose one model to train and then move on to the visualization of the training measures in the succeeding sections.

In [ ]:
# No class weights, no SGDR
history = model.fit_generator(train_generator,
                              train_generator.n // batch_size,
                              workers=4,
                              epochs=20,
                              callbacks=[mmetrics])

In [ ]:
# No class weights, SGDR
history = model.fit_generator(train_generator,
                              train_generator.n // batch_size,
                              workers=4,
                              epochs=len(sgdr_sched_epoch),
                              callbacks=[mmetrics, sgdr_sched_cb])

In [ ]:
# Class weights, no SGDR
history = model.fit_generator(train_generator,
                              train_generator.n // batch_size,
                              workers=4,
                              epochs=len(sgdr_sched_epoch),
                              class_weight=class_weight,
                              callbacks=[mmetrics])

In [ ]:
# Class weights, SGDR
history = model.fit_generator(train_generator,
                              train_generator.n // batch_size,
                              workers=4,
                              epochs=len(sgdr_sched_epoch),
                              class_weight=class_weight,
                              callbacks=[mmetrics, sgdr_sched_cb])

In [ ]:
#Standard
history = model.fit_generator(train_generator,
                              train_generator.n // batch_size,
                              validation_data=validation_generator,
                              validation_steps=validation_generator.n // batch_size,
                              workers=4,
                              epochs=20,
                              class_weight=class_weight,
                              callbacks=[mmetrics])

### Save the trained model

In [ ]:
model.save('incresnetv2_600_sgdr_1_10_epochs_all_final.h5')

## Visualization of training statistics

Visualizations of training and model performance statistics. Execute cells as appropriate for the trainined model.


### Sensitivity / specificity plot w/AUC

In [ ]:
sens = mmetrics.hist['sn']
spec = mmetrics.hist['sp']
avg = mmetrics.hist['snspavg']
roc_auc = mmetrics.hist['auc']
    
plt.plot(range(len(sens)), sens, 'b', label = 'Sn')
plt.plot(range(len(spec)), spec, 'r', label = 'Sp')
plt.plot(range(len(avg)), avg, 'bo', label = 'SnSp Avg')
plt.plot(range(len(roc_auc)), roc_auc, 'ro', label = 'AUC')
plt.xlabel('Epochs')
plt.title('Validation Results')
plt.legend(loc="lower right")

### Plot results

Run this cell before running any of the below visualizations.

In [ ]:
def plot_results(labels, predictions, thresh = 0.5):

    md = med_metrics(validation_generator.classes, predictions, thresh=thresh)
    sn = md['sn']
    sp = md['sp']
    snspavg = md['snspavg']
    roc_auc = md['auc']
    tpr = md['tpr']
    fpr = md['fpr']
    cm = md['cm']
    acc = md['acc']

    print('Results at threshold', str(thresh) + ':')
    print('Sn =', round(sn * 100, 1), '%')
    print('Sp =', round(sp * 100, 1), '%')
    print('Avg =', round(snspavg * 100, 1), '%')
    print('Acc =', round(acc * 100, 1), '%')
    print(cm)

    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")

### Validation w/test time image augmentation; sensitivity, specificity, accuracy

Test the model on the validation data set with test time image augmentation (TTA). This should provide a rough lower bound on the model's general purpose performance where the positioning and quality of the input images may not be as precise as those we used for training and validation.

In [ ]:
tta_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, 
    zoom_range=0.1,
    channel_shift_range=0.1,
    fill_mode='constant',
    cval=0)

tta_generator = tta_datagen.flow_from_directory(
    valid_dir,
    target_size=(height, width), 
    shuffle=False, 
    batch_size=batch_size,
    class_mode='binary',
    classes = ['norm', 'frac'])

In [ ]:
repeats = 5 # number of times to augment
preds = np.zeros((tta_generator.classes.shape[0], repeats))

for i in range(repeats):
    print('Predicting augmentation #', i + 1, '\r', end='')
    val_preds = model.predict_generator(tta_generator)
    preds[:, i] = val_preds[:, 0]

In [ ]:
max_preds = preds.max(axis=1)
mean_preds = preds.mean(axis=1)

In [ ]:
plot_results(validation_generator.classes, max_preds, thresh=0.5)

In [ ]:
plot_results(validation_generator.classes, mean_preds, thresh=0.5)

### Sensitivity and specificity on validation data w/o TTA

In [ ]:
val_preds = model.predict_generator(validation_generator)

In [ ]:
plot_results(validation_generator.classes, val_preds, thresh=0.5)

### Save test results

In [ ]:
test_fnames = validation_generator.filenames
test_fnames = np.asarray(test_fnames)

np.save('test_fnames', test_fnames)
np.save('test_classes', validation_generator.classes)
np.save('test_preds', val_preds)
np.save('test_max_preds', max_preds)
np.save('test_mean_preds', mean_preds)

### Export results to Microsoft Excel

In [ ]:
test_fnames = np.load('test_fnames.npy')

for i in range(len(test_fnames)):
    test_fnames[i] = test_fnames[i][16:35]
    
test_classes = np.load('test_classes.npy')
test_preds = np.load('test_preds.npy')
test_max_preds = np.load('test_max_preds.npy')
test_mean_preds = np.load('test_mean_preds.npy')

In [ ]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
 
df = pd.DataFrame({'filenames': test_fnames,
                   'labels': test_classes,
                   'raw predictions': test_preds[:, 0],
                   'max augmented predictions': test_max_preds,
                   'mean augmented predictions': test_mean_preds
                  })
 
writer = ExcelWriter('final_results.xlsx')
df.to_excel(writer, 'Sheet1', index=False)
writer.save()

### Accuracy and loss charts when not using medical metrics callback

In [ ]:
acc = history.history['acc']
loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']

plt.plot(range(len(acc)), val_acc, 'b')
plt.plot(range(len(acc)), acc, 'bo')
plt.title('Training and validation accuracy')

In [ ]:
plt.plot(range(len(acc)), val_loss, 'b')
plt.plot(range(len(acc)), loss, 'bo')
plt.title('Training and validation loss')

## Learning Rate Finder

Code to find the optimal learning rate using a small sample of images. The optimal learning rate can then be used in prior cells for training on the full data set.

In [ ]:
model.save('temp.h5')

In [ ]:
train_dir_lr = os.path.join(base, 'lr_finder')

In [ ]:
train_datagen_lr = ImageDataGenerator(rescale = 1. / 255)
train_generator_lr = train_datagen_lr.flow_from_directory(train_dir_lr,
    class_mode='binary',
    target_size=(height,width),
    batch_size=batch_size)

finder_steps = train_generator_lr.n // batch_size
epochs = 50 # number of learning rates to check

In [ ]:
# Create a schedule of increasing learning rates
def lr_finder_sched(steps, base_lr = 10**-10):
  finder_sched = np.ones((steps))
  finder_sched = finder_sched * base_lr
  exponent = np.arange(0, 10, 10/steps)
  finder_sched = finder_sched * 10 ** exponent
  return finder_sched

In [ ]:
lr_sched = lr_finder_sched(epochs)

In [ ]:
plt.plot(range(len(lr_sched)), lr_sched)

In [ ]:
# Returns the learning rate for a given epoch
def lr_returner(epoch):
    return lr_sched[epoch]

In [ ]:
lrate = callbacks.LearningRateScheduler(lr_returner, verbose=1)

In [ ]:
train_labels, train_counts = np.unique(train_generator.classes, return_counts=True)
class_weight = {0: train_counts[1] / train_counts[0], 1: 1.}

In [ ]:
model.compile(optimizer=optimizers.Adam(lr=0), loss='binary_crossentropy', metrics=['acc'])
history = model.fit_generator(train_generator_lr,
                              steps_per_epoch=finder_steps,
                              epochs=epochs,
                              class_weight=class_weight,
                              callbacks=[lrate])

In [ ]:
plt.loglog(lr_sched, history.history['loss'])

In [ ]:
model = load_model('temp.h5')

## Convolational neural network class activation heatmap

Generates heatmaps that show the areas of an image that maximize a trained model.

In [ ]:
alpha = 0.3 # for transparency
image_source = os.path.join(valid_dir, 'norm')
heatmap_path = os.path.join(base, 'heatmaps')
fnames = os.listdir(image_source)
starting_index = 1980

for ind in range(starting_index, len(fnames)):
    
    print("Processing heatmap", ind, 'of', len(fnames), '\r', end='')

    img = image.load_img(os.path.join(image_source,fnames[ind]), target_size=(height, width))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.

    pred = model.predict(x)
    frac_output = model.output[:, 0]
    last_conv_layer = model.layers[0]

    grads = K.gradients(frac_output, last_conv_layer.get_output_at(1))[0]

    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.get_output_at(1)[0]])
    pooled_grads_value, conv_layer_output_value = iterate([x])

    for i in range(1536):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    img = cv2.imread(os.path.join(image_source,fnames[ind]))
    
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap_fname = 'heatmap_' + '%03d' % int(pred * 100) + '_' + fnames[ind]
    heatmap = cv2.addWeighted(heatmap, alpha, img, 1 - alpha, 0)

    cv2.imwrite(os.path.join(heatmap_path, heatmap_fname), heatmap)

## Best Model Characteristics

sgdr_sched_epoch = cos_sgdr_sched(10,1,2) at 0.0001 base lr

600 resolution, TTA * 4

Raw results:
Sn = 82.0 %
Sp = 94.5 %
Avg = 88.25 %

Mean augmented results:
Sn = 81.3 %
Sp = 96.1 %
Avg = 88.7 %

Minpooled augmented results:
Sn = 88.7 %
Sp = 89.8 %
Avg = 89.25 %

*with BN

Raw results:
Sn = 84.0 %
Sp = 93.8 %
Avg = 88.9 %

Minpooled augmented results:
Sn = 88.7 %
Sp = 86.2 %
Avg = 87.45 %

Mean augmented results:
Sn = 84.0 %
Sp = 94.0 %
Avg = 89.0 %

